In [34]:
from sentence_transformers import SentenceTransformer
import csv
import pandas as pd
from sklearn.mixture import GaussianMixture as GMM
import os

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [35]:
with open('../preprocessed_comments/CNN-Full Speech: President Biden’s 2024 State of the Union address_liked.csv', 'r') as f:
    df = pd.read_csv(f)
df

,comment,likes_count
0,you cant love your country only when you win,1627
1,you cant only love your country when you win t...,1050
2,the number of times mike johnson nodded in agr...,702
3,dont pay congress until they do their job,650
4,unions build middle class not billionaires ind...,543
...,...,...
2870,they pumped him full of dementia medication so...,1
2871,well that was certainly revealing if i ever do...,1
2872,i wouldnt be mentioning let alone boasting abo...,1
2873,NaN,1


In [36]:
# Downloads the transformer to your device. After the first time running will load it from device
model = SentenceTransformer('all-MiniLM-L6-v2')

In [37]:
# remove nan values
df = df.dropna()

In [38]:
embeddings = df['comment'].map(lambda x: model.encode(x))

In [39]:
# converts embeddings into a dataframe
embeddings = embeddings.to_frame()
embeddings = embeddings['comment'].apply(pd.Series)

In [40]:
embeddings_df = pd.DataFrame(embeddings)

In [41]:
# fit the GMM model
gmm_10 = GMM(n_components=10, covariance_type='diag', max_iter=2000, random_state=0).fit(embeddings_df)
gmm_5 = GMM(n_components=5, covariance_type='diag', max_iter=2000, random_state=0).fit(embeddings_df)
gmm_2 = GMM(n_components=2, covariance_type='diag', max_iter=2000, random_state=0).fit(embeddings_df)
# Predict the labels for the data samples
labels_10 = gmm_10.predict(embeddings_df)
labels_5 = gmm_5.predict(embeddings_df)
labels_2 = gmm_2.predict(embeddings_df)

In [42]:
df_10 = df.copy()
df_5 = df.copy()
df_2 = df.copy()
# Save data frame with labels
df_10['label'] = labels_10
df_5['label'] = labels_5
df_2['label'] = labels_2

# Sort by label
df_10 = df_10.sort_values(by='label')
df_5 = df_5.sort_values(by='label')
df_2 = df_2.sort_values(by='label')

In [44]:
file_name = 'CNN-Full Speech: President Biden’s 2024 State of the Union address_liked'
df_2.to_csv('../clustered_data/cluster/' + file_name + '_2.csv', index=False)
df_5.to_csv('../clustered_data/cluster/' + file_name + '_5.csv', index=False)
df_10.to_csv('../clustered_data/cluster/' + file_name + '_10.csv', index=False)

In [47]:
# Save the model

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'